This Kernel for someone want to deep dive into image classification. I use CNN for classification model. If you found this Kernel helpful please up vote it. If you have some feedback and question don't forget to comment below. 

I have simplier model with 
* https://www.kaggle.com/uysimty/get-start-image-classification

# Import Library

In [1]:
import numpy as np
import pandas as pd 
from keras.preprocessing.image import ImageDataGenerator # 이미지 데이터의 배치를 생성, 이미지 전처리에 사용
from keras.utils import to_categorical # 여러 개의 Y값을 0과 1로만 이루어진 형태로 바꿔주는 one-hot-encoding
from sklearn.model_selection import train_test_split # scikit-learn 패키지 중 model_selection에 데이터 분할을 위한 train_test_split 함수가 있음.
import matplotlib.pyplot as plt
import random
import os # os.listdir을 쓰기 위해서


In [2]:
import urllib
import optuna
from optuna.integration import TFKerasPruningCallback
from optuna.trial import TrialState

import tensorflow as tf

/Users/jeong-geun-o/miniconda3/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Define Constants

In [3]:
opener = urllib.request.build_opener()
opener.addheaders = [("User-agent", "Mozilla/5.0")]
urllib.request.install_opener(opener)

In [3]:
FAST_RUN = False # 아래에 사용됨
IMAGE_WIDTH=128 # 이미지 넓이(행)
IMAGE_HEIGHT=128 # 이미지 높이(열)
IMAGE_SIZE=(IMAGE_WIDTH, IMAGE_HEIGHT)
IMAGE_CHANNELS=3 # 이미지 channel(depth)

In [4]:
BATCHSIZE = 256
CLASSES = 2
EPOCHS = 7
N_TRAIN_EXAMPLES = 3000
STEPS_PER_EPOCH = int(N_TRAIN_EXAMPLES / BATCHSIZE)
VALIDATION_STEPS = 30

In [5]:
global train_df
global validate_df

# Prepare Traning Data

In [7]:
# 케글 데이터 다운받기
import os

# os.environ을 이용하여 Kaggle API Username, Key 세팅하기
# os.environ['KAGGLE_USERNAME'] = ''
# os.environ['KAGGLE_KEY']=''

# 데이터셋 다운로드 

!kaggle competitions download -c dogs-vs-cats

100%|███████████████████████████████████████▊| 809M/812M [00:41<00:00, 39.0MB/s]
100%|████████████████████████████████████████| 812M/812M [00:41<00:00, 20.7MB/s]


In [ ]:
!unzip '*.zip' 
!unzip 'train.zip' 
!unzip 'test1.zip' 

In [62]:
import numpy as np
import pandas as pd
import glob


def train_set_name_changer():
    train_files = [i[6:] for i in glob.glob('train/*.jpg')]
    train_labels = [i[6:9] for i in glob.glob('train/*.jpg')]

    df = pd.DataFrame({
        'filename':train_files,
        'category':train_labels
    })

train_set_name_changer()

# Prepare data

Because we will use image genaretor `with class_mode="categorical"`. We need to convert column category into string. Then imagenerator will convert it one-hot encoding which is good for our classification. 

So we will convert 1 to dog and 0 to cat

In [38]:
df.head() # category 확인

,filename,category
0,dog.8011.jpg,dog
1,cat.5077.jpg,cat
2,dog.7322.jpg,dog
3,cat.2718.jpg,cat
4,cat.10151.jpg,cat


In [39]:
train_df, validate_df = train_test_split(df, test_size=0.20, random_state=42)
# df: 분할시킬 data (DataFrame)
# test_size: test dataset의 비율이나 갯수(default=0.25) -> 여기서는 validation set이라 할 수 있다.
# random state: 데이터 분할시 셔플이 이루어지는데 이를 위한 시드값
# shuffle: 셔플여부설정 (default=True)

train_df = train_df.reset_index(drop=True) # index를 reset함 -> drop: index로 세팅한 열을 데이터프레임내에서 삭제할지 여부를 결정한다.
validate_df = validate_df.reset_index(drop=True)

In [40]:
total_train = train_df.shape[0]
total_validate = validate_df.shape[0]
batch_size=15

In [41]:
# 제가 뭔지 확인해 본거에요.
print(train_df.shape) 
print(validate_df.shape) 

(20000, 2)
(5000, 2)


In [42]:
train_df

,filename,category
0,cat.11282.jpg,cat
1,cat.7693.jpg,cat
2,cat.4977.jpg,cat
3,dog.9403.jpg,dog
4,cat.10150.jpg,cat
...,...,...
19995,dog.2721.jpg,dog
19996,dog.3974.jpg,dog
19997,dog.10427.jpg,dog
19998,cat.731.jpg,cat


# Traning Generator

In [45]:
def train_dataset(train_df):
    # 신경망 모델의 성능을 높이기 위한 위한 "데이터 부풀리기(Data augmentation)"
    train_datagen = ImageDataGenerator(
        rotation_range=15,
        rescale=1./255,
        shear_range=0.1,
        zoom_range=0.2,
        horizontal_flip=True,
        width_shift_range=0.1,
        height_shift_range=0.1
    )
    # rotation_range : 지정된 각도 "범위"내에서 임의로 원본이미지를 회전시킨다. // 단위:(도) -> 여기서는 15니까, 0도에서 15도 사이
    # rescale : byte와 관련이 있는듯 255라는 숫자가
    # shear_range : 밀림 강도 범위내에서 임의로 원본이미지를 변형시킴

    # zoom_range : 지정된 확대/축소 범위내에서 임의로 원본이미지를 확대/축소. “1 - 수치”부터 “1 + 수치”사이 범위로 확대/축소.
    #              예를 들어 0.2이라면, 0.8배에서 1.2배 크기 변화를 시킵니다.

    # horizontal_flip : 수평방향으로 뒤집느냐 마느냐

    # width_shift_range : 지정된 "수평방향" 이동 범위내에서 임의로 원본이미지를 이동시킴. 수치는 "전체 넓이"의 비율(실수)로 나타냄.
    #                     예를 들어 0.1이고 전체 넓이가 100이면, 10픽셀 내외로 "좌우" 이동.

    # height_shift_range : 지정된 "수직방향" 이동 범위내에서 임의로 원본이미지를 이동시킴. 수치는 "전체 높이"의 비율(실수)로 나타냄..
    #                       예를 들어 0.1이고 전체 높이가 100이면, 10픽셀 내외로 "상하" 이동.


    # 'dataframe'과 '디렉토리'의 위치를 전달받아 증강/정규화된 데이터의 "배치"를 생성.
    train_generator = train_datagen.flow_from_dataframe( 
        train_df, # DataFrame (전체 중에 80%만 있음)
        "train/", # 문자열, 이미지를 읽을 '디렉토리'의 경로 (전체 data가 있음) 
        x_col='filename', # train_df 데이터프레임에 filename col의 값들을 가져옴
        y_col='category', # train_df 데이터프레임에 category col의 값들을 가져옴
        target_size=IMAGE_SIZE, # 
        class_mode='categorical', # 2D numpy array of one-hot encoded labels. Supports multi-label output.
        batch_size=batch_size # batch size
    )
    return train_generator

In [46]:
train_ds = train_dataset(train_df)
train_ds

Found 20000 validated image filenames belonging to 2 classes.


### Validation Generator

In [47]:
def validate_dataset(validate_df):
    # 위에 train_datagen과는 다르게 rescale만 해줌. (검증하는 거니까!)
    # rescale 하는 이유: 정규화 과정임. image가 0~255까지 값을 가지는 2차원 배열인데, 0~255 사이의 값을
    #                   0.0과 1.0사이의 값으로 바꾸기 위함이다.
    #                   활성화함수 및 오류역전파 알고리즘은 0.0과 1.0사이의 값을 좋아하기 때문이다.
    validation_datagen = ImageDataGenerator(rescale=1./255)

    # 위에 있는 train_generator와 같다
    validation_generator = validation_datagen.flow_from_dataframe(
        validate_df, 
        "train/", 
        x_col='filename',
        y_col='category',
        target_size=IMAGE_SIZE,
        class_mode='categorical',
        batch_size=batch_size
    )

    return validation_generator 

In [48]:
validate_ds = validate_dataset(validate_df)
validate_ds

Found 5000 validated image filenames belonging to 2 classes.


Seem to be nice 

* **Input Layer**: It represent input image data. It will reshape image into single diminsion array. Example your image is 64x64 = 4096, it will convert to (4096,1) array.
* **Conv Layer**: This layer will extract features from image.
* **Pooling Layer**: This layerreduce the spatial volume of input image after convolution.
* **Fully Connected Layer**: It connect the network from a layer to another layer
* **Output Layer**: It is the predicted values layer. 

In [49]:
from keras.applications import VGG16
from keras.applications.resnet import ResNet50

def create_model(trial):
    # Hyperparameters to be tuned by Optuna.
    learning_rate = trial.suggest_float("learning_rate", 1e-4, 1e-1, log=True)
    momentum = trial.suggest_float("momentum", 0.0, 1.0)
    units = trial.suggest_categorical("units", [32, 64, 128, 256, 512])
    
    conv_base = ResNet50(weights='imagenet',
                    include_top=False,
                    input_shape=(IMAGE_WIDTH, IMAGE_HEIGHT, IMAGE_CHANNELS))

    model = tf.keras.Sequential() # 가장 자주 사용하는 구조인 층을 순서대로 쌓아 올린 네트워크{keras 사용 - 모델을 정의(생성)}
    conv_base.trainable = False  # Freeze weights of conv_base layers (VGG16)

    # Convolution
    # 인공신경망 모델을 효율적으로 학습시키기 윈한 개선 방법들 (BatchNormalization, Dropout, ModelEnsemble)
    model.add(conv_base)
    model.add(tf.keras.layers.Flatten()) # - 이 층에는 학습되는 가중치가 없고 데이터를 변환하기만 한다.(1차원 vector으로 바꿈)
                        # - 이 층은 하나의 layer에 있는 모든 neuron을 또 다른 layer의 모든 neuron과 연결 시켜준다.
    model.add(tf.keras.layers.Dense(256, activation='relu')) # dense로 층을 쌓음 - 첫번째 Dense층은 512개의 노드를 가짐
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Dropout(0.5))
    model.add(tf.keras.layers.Dense(2, activation='softmax')) # 2: because we have cat and dog classes
                                            # 마지막 층은 2개의 노드의 소프트맥스층 : 이 층은 2개의 확률을 반환하고 반환된 값의 전체 합은 1이다. 
                                            # 각 노드는 현재 이미지가 2개 클래스 중 하나에 속할 확률을 출력

    # Compile model.
    model.compile(
        optimizer=tf.keras.optimizers.SGD(
            learning_rate=learning_rate, momentum=momentum, nesterov=True
        ),
        loss="categorical_crossentropy",
        metrics=["accuracy"],
    )

    return model

In [55]:
def objective(trial):
    global train_df
    global validate_df
    # Clear clutter from previous TensorFlow graphs.
    tf.keras.backend.clear_session()

    # Metrics to be monitored by Optuna.
    if tf.__version__ >= "2":
        monitor = "val_accuracy"
    else:
        monitor = "val_acc"

    # Create tf.keras model instance.
    model = create_model(trial)

    # Create dataset instance.
    ds_train = train_dataset(train_df)
    ds_eval = validate_dataset(validate_df)

    # Create callbacks for early stopping and pruning.
    callbacks = [
        tf.keras.callbacks.EarlyStopping(patience=3),
        TFKerasPruningCallback(trial, monitor),
    ]

    # Train model.
    history = model.fit(
        ds_train,
        epochs=EPOCHS,
        steps_per_epoch=STEPS_PER_EPOCH,
        validation_data=ds_eval,
        validation_steps=VALIDATION_STEPS,
        callbacks=callbacks,
    )
    model.save_weights("res.h5") 
    return history.history[monitor][-1]

In [51]:
def show_result(study):

    pruned_trials = study.get_trials(deepcopy=False, states=[TrialState.PRUNED])
    complete_trials = study.get_trials(deepcopy=False, states=[TrialState.COMPLETE])

    print("Study statistics: ")
    print("  Number of finished trials: ", len(study.trials))
    print("  Number of pruned trials: ", len(pruned_trials))
    print("  Number of complete trials: ", len(complete_trials))

    print("Best trial:")
    trial = study.best_trial

    print("  Value: ", trial.value)

    print("  Params: ")
    for key, value in trial.params.items():
        print("    {}: {}".format(key, value))
    
    return 

In [52]:
def main():

    study = optuna.create_study(
        direction="maximize", pruner=optuna.pruners.MedianPruner(n_startup_trials=2)
    )

    study.optimize(objective, n_trials=trials, timeout=600)

    show_result(study)

In [56]:
trials = 3

if __name__ == "__main__":
    main()

[I 2023-01-18 18:35:16,451] A new study created in memory with name: no-name-605222d1-1600-405c-86cc-62813ad6d0f7


Found 20000 validated image filenames belonging to 2 classes.
Found 5000 validated image filenames belonging to 2 classes.
Epoch 1/7


2023-01-18 18:35:20.776040: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


11/11 [==============================] - ETA: 0s - loss: 1.1112 - accuracy: 0.6182

2023-01-18 18:35:24.459785: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


...

11/11 [==============================] - ETA: 0s - loss: 0.8520 - accuracy: 0.5576

2023-01-18 18:36:19.828802: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


11/11 [==============================] - 9s 636ms/step - loss: 0.8520 - accuracy: 0.5576 - val_loss: 0.6819 - val_accuracy: 0.5822
Epoch 2/7
11/11 [==============================] - 4s 323ms/step - loss: 0.9879 - accuracy: 0.5576 - val_loss: 0.6906 - val_accuracy: 0.5267
Epoch 3/7
11/11 [==============================] - 3s 306ms/step - loss: 0.9562 - accuracy: 0.5091 - val_loss: 0.7614 - val_accuracy: 0.5156
Epoch 4/7
11/11 [==============================] - 3s 303ms/step - loss: 0.7785 - accuracy: 0.5879 - val_loss: 0.7056 - val_accuracy: 0.5111


[I 2023-01-18 18:36:34,372] Trial 2 finished with value: 0.5111111402511597 and parameters: {'learning_rate': 0.0007890436524346838, 'momentum': 0.23940193826459555, 'units': 64}. Best is trial 2 with value: 0.5111111402511597.


Study statistics: 
  Number of finished trials:  3
  Number of pruned trials:  0
  Number of complete trials:  3
Best trial:
  Value:  0.5111111402511597
  Params: 
    learning_rate: 0.0007890436524346838
    momentum: 0.23940193826459555
    units: 64


In [71]:
optuna.get_all_study_summaries()

TypeError: get_all_study_summaries() missing 1 required positional argument: 'storage'

# Save Model

In [63]:
test_filenames = os.listdir("test1") # test에 있는 list들을 가져옴
test_df = pd.DataFrame({                                       # 데이터프레임을 생성
    'filename': test_filenames                                 # filename이라는 col에 test_filenames값을 가져옴
})
nb_samples = test_df.shape[0] # test dataset data 갯수

In [66]:
test_df

,filename
0,9733.jpg
1,63.jpg
2,6400.jpg
3,823.jpg
4,4217.jpg
...,...
12495,3561.jpg
12496,8434.jpg
12497,7707.jpg
12498,6419.jpg


In [64]:
test_df.shape

(12500, 1)

# Create Testing Generator

In [67]:
# rescale만 해줌. (검증하는 거니까!)
# rescale 하는 이유: 정규화 과정임. image가 0~255까지 값을 가지는 2차원 배열인데, 0~255 사이의 값을
#                   0.0과 1.0사이의 값으로 바꾸기 위함이다.
#                   활성화함수 및 오류역전파 알고리즘은 0.0과 1.0사이의 값을 좋아하기 때문이다.
test_gen = ImageDataGenerator(rescale=1./255)

test_generator = test_gen.flow_from_dataframe(
    test_df, 
    "test1/", 
    x_col='filename',
    y_col=None,
    class_mode=None,
    target_size=IMAGE_SIZE,
    batch_size=batch_size,
    shuffle=False
)

# shuffle을 쓰지 않음

Found 12500 validated image filenames.


# Predict

In [74]:
def show_result(study):

    pruned_trials = study.get_trials(deepcopy=False, states=[TrialState.PRUNED])
    complete_trials = study.get_trials(deepcopy=False, states=[TrialState.COMPLETE])

    print("Study statistics: ")
    print("  Number of finished trials: ", len(study.trials))
    print("  Number of pruned trials: ", len(pruned_trials))
    print("  Number of complete trials: ", len(complete_trials))

    print("Best trial:")
    trial = study.best_trial

    print("  Value: ", trial.value)

    print("  Params: ")
    for key, value in trial.params.items():
        print("    {}: {}".format(key, value))

In [68]:
predict = model.predict_generator(test_generator, steps=np.ceil(nb_samples/batch_size))
# test_generator 자리 : 입력 샘플의 batch를 생성하는 생성기
# steps : 중지되기 전까지 generator로부터 얻는 단계의 총 갯수 (샘플의 batch) , np.ceil로 '올림'함


NameError: name 'model' is not defined

For categoral classication the prediction will come with probability of each category. So we will pick the category that have the highest probability with numpy average max

In [ ]:
predict

In [ ]:
np.argmax(predict, axis=-1)

In [ ]:
test_df['category'] = np.argmax(predict, axis=-1) # axis에 해당하는 값들 중 가장 큰 값의 인덱스들을 반환하는 함수

In [ ]:
test_df['category']

We will convert the predict category back into our generator classes by using `train_generator.class_indices`. It is the classes that image generator map while converting data into computer vision

In [ ]:
dict((v,k) for k,v in train_generator.class_indices.items())

In [ ]:
label_map = dict((v,k) for k,v in train_generator.class_indices.items())
test_df['category'] = test_df['category'].replace(label_map) # test_df['category'] 값을 0 or 1을 cat or dog로 바꿈

From our prepare data part. We map data with `{1: 'dog', 0: 'cat'}`. Now we will map the result back to dog is 1 and cat is 0

In [ ]:
test_df

In [ ]:
test_df['category'] = test_df['category'].replace({ 'dog': 1, 'cat': 0 })

In [ ]:
test_df